# Examples for the DLA Survey Class (v2.0)

In [1]:
%matplotlib notebook

In [2]:
# imports
import imp, os
from imp import reload
import numpy as np
import pdb

import datetime

from pkg_resources import resource_filename

from matplotlib import pyplot as plt

from astropy.table import Table

from linetools.isgm import utils as ltiu
from linetools.lists.linelist import LineList

from pyigm.surveys.dlasurvey import DLASurvey
from pyigm.surveys import dlasurvey as dlasrv
from pyigm.abssys.igmsys import AbsSubSystem

read_sets: Using set file -- 
  /home/xavier/local/Python/linetools/linetools/lists/sets/llist_v1.2.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [3]:
ism = LineList('ISM')

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


## Instantiate

In [4]:
dlas = DLASurvey(ref='null')

### H100  (Neeleman, M. et al. 2013, ApJ, 769, 54)

In [5]:
reload(dlasrv)
print(datetime.datetime.now())
h100 = dlasrv.DLASurvey.load_H100()
print(datetime.datetime.now())
#h100

2018-01-10 16:23:21.940122
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
H100: Loading summary file /home/xavier/local/Python/pyigm/pyigm/data/DLA/H100/H100_DLA.fits
H100: Loading ions file /home/xavier/local/Python/pyigm/pyigm/data/DLA/H100/H100_DLA_ions.json
H100: Loading systems.  This takes ~90s
Loading systems from /home/xavier/local/Python/pyigm/pyigm/data/DLA/H100/H100_DLA_sys.tar.gz
Skipping a likely folder: SYS
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Starting the AbsSystem build.  Be patient..


/home/xavier/local/Python/linetools/linetools/spectralline.py:215: UserWarning: Redshift not input.  Setting to 0 for zLimits
  warnings.warn("Redshift not input.  Setting to 0 for zLimits")
/home/xavier/local/Python/linetools/linetools/spectralline.py:119: UserWarning: Key mol not in your input dict
  warnings.warn("Key {:s} not in your input dict".format(key))


Done!
Building the data Table from the internal dict
Filling the _ionN tables...
All done!!
2018-01-10 16:23:31.046069


In [6]:
h100.abs_type

'DLA'

#### Grab spectra (be patient; may require Internet access)

In [7]:
h100 = DLASurvey.load_H100(grab_spectra=True)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
H100: Loading summary file /home/xavier/local/Python/pyigm/pyigm/data/DLA/H100/H100_DLA.fits
H100: Loading ions file /home/xavier/local/Python/pyigm/pyigm/data/DLA/H100/H100_DLA_ions.json
H100: Loading systems.  This takes ~90s
Loading systems from /home/xavier/local/Python/pyigm/pyigm/data/DLA/H100/H100_DLA_sys.tar.gz
Skipping a likely folder: SYS
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Starting the AbsSystem build.  Be patient..


/home/xavier/local/Python/linetools/linetools/spectralline.py:215: UserWarning: Redshift not input.  Setting to 0 for zLimits
  warnings.warn("Redshift not input.  Setting to 0 for zLimits")
/home/xavier/local/Python/linetools/linetools/spectralline.py:119: UserWarning: Key mol not in your input dict
  warnings.warn("Key {:s} not in your input dict".format(key))


Done!
Building the data Table from the internal dict
Filling the _ionN tables...
H100: Downloading a 141Mb file.  Be patient..


/home/xavier/local/Python/pyigm/pyigm/surveys/dlasurvey.py:233: UserWarning: All of these spectra are in igmspec at https://github.com/specdb/specdb
  warnings.warn("All of these spectra are in igmspec at https://github.com/specdb/specdb")


HTTPError: HTTP Error 404: Not Found

#### Metallicity

In [ ]:
h100.ZH

## Simple Attributes

In [ ]:
h100.nsys

In [ ]:
h100.NHI[0:10]

In [ ]:
h100.name[0:5]

## Column Densities

In [ ]:
SiII_clms = h100.ions((14,2))
SiII_clms[0:5]

In [ ]:
gdSiII = SiII_clms['flag_N']>0
np.sum(gdSiII)

In [ ]:
h100.name[~gdSiII]

## Simple Plots

In [ ]:
# NHI
p_NHI = figure(plot_width=400, plot_height=400, title='H100 NHI')#,background_fill="#E8DDCB")
hist, edges = np.histogram(h100.NHI, range=(20.2, 22), density=True, bins=20)
p_NHI.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color='blue')
p_NHI.xaxis.axis_label = 'N_HI'
# z
p_z = figure(plot_width=400, plot_height=400, title='H100 z')#,background_fill="#E8DDCB")
hist, edges = np.histogram(h100.zabs, range=(1.6, 5.0), density=True, bins=20)
p_z.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color='red')
p_z.xaxis.axis_label = 'z_DLA'
# Show
show(row(p_NHI,p_z))

----

## SDSS DLAs (DR5) -- Prochaska & Wolfe 2009, ApJ, 696, 1543

### Load

In [ ]:
sdss = DLASurvey.load_SDSS_DR5(sample='all')
sdss

In [ ]:
sdss_stat = DLASurvey.load_SDSS_DR5()
sdss_stat

In [ ]:
sdss_stat.sightlines[0:5]

### $g(z)$

In [ ]:
zeval, gz = sdss_stat.calculate_gz()

In [ ]:
plt.clf()
plt.plot(zeval, gz)
plt.xlabel('z')
plt.ylabel('g(z)')
plt.show()

### Binned stats

#### $\ell(X)$

In [ ]:
lX, lX_lo, lX_hi= sdss_stat.binned_lox([2., 2.5, 3])

In [ ]:
lX, lX_lo, lX_hi

#### $\rho_{\rm HI}$

In [ ]:
rho, rho_lo, rho_hi = sdss_stat.binned_rhoHI([2., 2.5, 3.], (20.3,23.))

In [ ]:
rho/1e8, rho_lo, rho_hi

#### $f(N,X)$

In [ ]:
fN, fN_lo, fN_hi = sdss_stat.binned_fn([20.3, 20.5, 21., 21.5, 22.], [2, 2.5], log=True)

In [ ]:
fN, fN_lo, fN_hi

### Peroux03  (Peroux et al. 2003, MNRAS, 346, 1103)

In [ ]:
p03_statdla = DLASurvey.load_P03()#sample='all')
p03_statdla

### Guimaraes et al. 2009

In [ ]:
g09_statdla = DLASurvey.load_G09()#sample='all')
g09_statdla

In [ ]:
zeval, gz = g09_statdla.calculate_gz()

In [ ]:
plt.clf()
plt.plot(zeval, gz)
plt.xlabel('z')
plt.ylabel('g(z)')
plt.show()
plt.close()

## GGG (Crighton et al. 2015, MNRAS, 452, 217)

In [ ]:
ggg = DLASurvey.load_GGG()

In [ ]:
ggg

### XQ-100 (Sanchez-Ramirez et al. 2016, MNRAS, 456, 4488)

In [ ]:
xq100_statdla = DLASurvey.load_XQ100()
xq100_statdla

#### $g(z)$

In [ ]:
zeval, gz = xq100_statdla.calculate_gz()

In [ ]:
plt.clf()
plt.plot(zeval, gz)
plt.xlabel('z')
plt.ylabel('g(z)')
plt.show()
plt.close()

## HST -- (Neeleman et al. 2016)

In [ ]:
hst16 = DLASurvey.load_HST16()
hst16

In [ ]:
hst16_all = DLASurvey.load_HST16(sample='all')  # Includes MgII-selected DLAs
hst16_all

## Fitted quantities

In [ ]:
dlas = DLASurvey()

In [ ]:
# f(N)
dlas.fitted_fN(21.)  # This is not currently normalized

In [ ]:
# l(z)
dlas.fitted_lz(1.)

----

## Testing

In [1]:
tmp = [1,232,3,4]

In [2]:
tmp.index(5)

ValueError: 5 is not in list